In [18]:
# %load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 851 µs


In [19]:
# import sys
# sys.path.append('/Users/pradap/Documents/Research/Python-Package/enrique/')
# sys.path.append('/scratch/pradap/python-work/enrique')

time: 1.26 ms


In [20]:
import magellan as mg
import pandas as pd

time: 857 µs


In [21]:
# initialize JVM
mg.init_jvm()

True

time: 3.03 ms


In [22]:
# Read walmart books data
wal =  mg.read_csv(mg.get_install_path()+'/datasets/books/walmart.csv',
                    dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                    low_memory=False, key='id')

time: 1.36 s


In [23]:
# Read bowker books data
bwk = mg.read_csv(mg.get_install_path()+'/datasets/books/bowker.csv', 
                  dtype={'isbn':pd.np.str, 'pages':pd.np.str, 'volume':pd.np.str, 'editionNum':pd.np.str},  
                  low_memory=False, key='id')


time: 59.1 ms


In [24]:
# Create Attribute equivalence blocker and block using isbn
ab = mg.AttrEquivalenceBlocker()
C = ab.block_tables(wal, bwk, 'isbn', 'isbn', ['title', 'author'], ['title', 'author'])


time: 189 ms


In [25]:
# Write candidate set to disk -- end of project stage 2
C.to_csv('books_candset.csv')

True

time: 2.83 ms


In [87]:
# Read candidate set 
C = mg.read_csv('books_candset.csv', ltable=wal, rtable=bwk)

time: 4.42 ms


In [89]:
# print number of tuples in candidate set
len(C)

68

time: 2.18 ms


In [88]:
#plan

# 1. sample candidate set --> S
# 2. label S --> L
# 3. Two matching options
#    a. ML matcher + Trigger
#    b. Rule-based matcher + Trigger
# 4. Choose one of the above two options (which ever produces better result), and 
#     apply it over candidate set

time: 1.91 ms


In [90]:
# sample table
S = mg.sample_table(C, 50)

time: 2.27 ms


In [29]:
# label table
# L = mg.label_table(S, 'gold')

time: 636 µs


In [136]:
# for demo reading a pre-labeled file
L = mg.read_csv('label_table_demo_books.csv', ltable=wal, rtable=bwk)

time: 4.42 ms


In [31]:
# ML matcher + Trigger
#   1. Find best possible ML matcher
#       1. Get features for matching
#       2. Extract feature vectors
#       3. Select matcher using cross validation
#       4. Debug matcher
#       5. Get the best possible matcher
#   2. Add Triggers 

time: 1.17 ms


In [92]:
# get features for matching
feature_table = mg.get_features_for_matching(wal, bwk)

time: 3.04 s


In [33]:
#feature_table

time: 711 µs


In [93]:
# select a subset of matchers
f = feature_table.ix[[3,7,18,26, 53]]

time: 1.85 ms


In [94]:
# display selected features
f

,feature_name,left_attribute,right_attribute,left_attr_tokenizer,right_attr_tokenizer,simfunction,function,function_source
3,title_title_jac_qgm_3_qgm_3,title,title,qgm_3,qgm_3,jaccard,<function title_title_jac_qgm_3_qgm_3 at 0x120...,from magellan.feature.simfunctions import *\nf...
7,author_author_jac_qgm_3_qgm_3,author,author,qgm_3,qgm_3,jaccard,<function author_author_jac_qgm_3_qgm_3 at 0x1...,from magellan.feature.simfunctions import *\nf...
18,binding_binding_jac_qgm_3_qgm_3,binding,binding,qgm_3,qgm_3,jaccard,<function binding_binding_jac_qgm_3_qgm_3 at 0...,from magellan.feature.simfunctions import *\nf...
26,publisher_publisher_jac_qgm_3_qgm_3,publisher,publisher,qgm_3,qgm_3,jaccard,<function publisher_publisher_jac_qgm_3_qgm_3 ...,from magellan.feature.simfunctions import *\nf...
53,lang_lang_lev,lang,lang,None,None,lev,<function lang_lang_lev at 0x10dd13c08>,from magellan.feature.simfunctions import *\nf...


time: 17 ms


In [95]:
# Extract feature vectors
G = mg.extract_feat_vecs(L, feature_table=f, attrs_after='gold')

time: 325 ms


In [96]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,NaN,1,0.512821,1,0


time: 34.3 ms


In [97]:
# G contains NaN. 
# Impute values
G.fillna(0, inplace=True)

time: 2.05 ms


In [98]:
G.head(10)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
3,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1
4,4,15286798,9780217825382,1.000000,1.000000,1,1.000000,1,1
5,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0
6,6,15334872,9780217270052,1.000000,1.000000,1,1.000000,1,1
7,7,15338150,9780217304399,0.280702,1.000000,1,1.000000,1,1
8,8,1566948,9780070479081,0.822222,0.260870,1,0.303030,1,1
9,9,4564015,9780080446837,1.000000,0.000000,1,0.512821,1,0


time: 17.3 ms


In [99]:
# Create ML matchers
dt = mg.DTMatcher(name='DecisionTree', random_state=0)
svm = mg.SVMMatcher()
rf = mg.RFMatcher(name='RF', random_state=0)
nb = mg.NBMatcher(name='NB')
lg = mg.LogRegMatcher(name='LogReg')
ln = mg.LinRegMatcher(name='LinReg')

time: 3.66 ms


In [100]:
dt.name

'DecisionTree'

time: 1.9 ms


In [101]:
svm.name

'SVM_29108264476838216441'

time: 2.16 ms


In [102]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 
        # metric can take precision, recall, f1

time: 61.9 ms


In [44]:
# Check the selected matcher
result['selected_matcher']

time: 1.85 ms


In [45]:
# Display cross validation statistics
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.750000,1.000000,1.000000,0.833333,0.666667,0.850000
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.888889,1.000000,0.750000,0.777778,0.875000,0.858333
2,SVM_27205716404706216441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.888889,1.000000,0.700000,0.400000,0.857143,0.769206
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.700000,0.700000,0.666667,0.888889,0.800000,0.751111
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,1.000000,0.714286,0.555556,1.000000,0.666667,0.787302


time: 15.8 ms


In [46]:
# debug: understand why DT matcher is not performing well

# Plan : 
# Split data into 'train' and 'test'. 
# Train matcher using 'train' set
# Inspect matcher using 'test' set

time: 5 ms


In [103]:
# Split feature vectors to train and test
train_test = mg.train_test_split(G, train_proportion=0.7, random_state=0)
train = train_test['train']
test = train_test['test']

time: 4.24 ms


In [104]:
mg.vis_debug_dt(dt, train, test, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold') 

time: 2min 4s


In [108]:
# Experimental feature - requires installation of graphviz : www.graphviz.org
#mg.visualize_tree(dt, G.columns, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'])

time: 696 µs


In [107]:
#from IPython.display import Image
#Image(filename='dt_.png') 

time: 855 µs


In [109]:
mg.vis_debug_rf(rf, train, test, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold') 

time: 1min 22s


In [110]:
# Try adding all the features
H = mg.extract_feat_vecs(L, feature_table=feature_table, attrs_after='gold')

time: 561 ms


In [111]:
# select the best ML matcher using CV
result = mg.select_matcher([dt, rf, svm, nb, lg], table=G, 
        exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold', metric='precision') 

time: 67.3 ms


In [112]:
result['selected_matcher']

time: 2.51 ms


In [113]:
result['cv_stats']

,Name,Matcher,Num folds,Fold 1,Fold 2,Fold 3,Fold 4,Fold 5,Mean score
0,DecisionTree,<magellan.matcher.dtmatcher.DTMatcher object a...,5,0.888889,0.777778,1.000000,0.857143,0.666667,0.838095
1,RF,<magellan.matcher.rfmatcher.RFMatcher object a...,5,0.625000,0.875000,0.875000,1.000000,0.875000,0.850000
2,SVM_29108264476838216441,<magellan.matcher.svmmatcher.SVMMatcher object...,5,0.888889,0.700000,0.666667,0.666667,0.700000,0.724444
3,NB,<magellan.matcher.nbmatcher.NBMatcher object a...,5,0.700000,0.777778,0.800000,0.777778,0.700000,0.751111
4,LogReg,<magellan.matcher.logregmatcher.LogRegMatcher ...,5,0.666667,0.888889,0.875000,1.000000,0.700000,0.826111


time: 27.6 ms


In [55]:
# Choose DT matcher (with subset of features)

# Add triggers to DT Matcher
# Ideal plan
# Split G
# G :------------------|----------------------------|--------------------|
#   [development(60%): train(30%)  + validation(30%)]     [test(40%)]
# Use train + validation to come up with triggers
# Use test set to evaluate ML matcher + triggers


time: 1.68 ms


In [114]:
# Split G into development and test
development_test = mg.train_test_split(G, train_proportion=0.6, 
                                       random_state=0)
development = development_test['train']
test = development_test['test']

time: 4.39 ms


In [115]:
len(development), len(test)

(30, 20)

time: 2.56 ms


In [116]:
development.head(2)

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold
15,15,5087512,9780028348261,0.534483,0.6,1,0.264706,1,1
30,30,14022273,9780217263016,0.408759,1.0,1,1.000000,1,1


time: 12.4 ms


In [117]:
# First check, how does DT performs with out any triggers
# Train using development
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 7.24 ms


In [118]:
# Test using test
I = dt.predict(table=test, 
              exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)

time: 9.31 ms


In [119]:
# evaluate the result.
eval_summary = mg.eval_matches(I, 'gold', 'predicted')

time: 45.2 ms


In [120]:
eval_summary.keys()

['prec_numerator',
 'prec_denominator',
 'precision',
 'recall_numerator',
 'recall_denominator',
 'recall',
 'f1',
 'pred_pos_num',
 'false_pos_num',
 'false_pos_ls',
 'pred_neg_num',
 'false_neg_num',
 'false_neg_ls']

time: 2.64 ms


In [121]:
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.9333333333333333, 0.8235294117647058, 0.8749999999999999, 3.0, 1.0)

time: 2.45 ms


In [122]:
# Split development into train and validation set. Use them to debug.

train_validation = mg.train_test_split(development, train_proportion=0.5, random_state=10)
mg.vis_debug_dt(dt, train_validation['train'], train_validation['test'], 
               exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
               target_attr='gold')


time: 7.5 s


In [123]:
# Add triggers to address false negatives
# if authors match exactly then declare match
pos_trigger = mg.MatchTrigger()
pos_trigger.add_cond_rule('author_author_lev(ltuple, rtuple) == 1', 
                          feature_table=feature_table)
pos_trigger.add_cond_status(True)
pos_trigger.add_action(1)


True

time: 3.9 ms


In [124]:
# Apply ml matcher + trigger to 'development' set.
dt.fit(table=train_validation['train'], exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
       target_attr='gold')
J = dt.predict(table=train_validation['train'], exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'], 
              target_attr='predicted', append=True, inplace=False)

K = pos_trigger.execute(J, 'predicted', inplace=False)

time: 529 ms


In [125]:
K

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold,predicted
0,9,4564015,9780080446837,1.000000,0.000000,1,0.512821,1,0,0
1,39,15912308,9780077905972,0.709677,1.000000,0,0.200000,1,0,1
2,47,15359574,9780217298445,0.936170,1.000000,1,1.000000,1,1,1
3,19,22922094,9780077864262,0.015385,0.390244,0,0.281250,1,1,1
4,5,15295588,9780217940207,0.308271,0.695652,1,0.083333,1,0,0
5,3,15280600,9780217455008,0.720000,0.666667,1,1.000000,1,1,1
6,44,15289952,9780217048255,1.000000,1.000000,1,1.000000,1,1,1
7,13,7809485,9780152062279,0.739130,1.000000,1,0.547619,1,1,1
8,46,15358361,9780217249775,1.000000,1.000000,1,1.000000,1,0,1
9,37,15772172,9780061965548,1.000000,1.000000,1,0.320000,1,0,1


time: 36.9 ms


In [126]:
eval_summary = mg.eval_matches(K, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7272727272727273, 1.0, 0.8421052631578948, 0.0, 3.0)

time: 12.1 ms


In [128]:
#Apply DT matcher + positive trigger  over "test" set
dt.fit(table=development, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')
L_ = dt.predict(table=test, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
              target_attr='predicted',
              append=True, inplace=False)
M = pos_trigger.execute(L_, 'predicted', inplace=False)


time: 517 ms


In [129]:
# evaluate ML matcher + trigger
eval_summary = mg.eval_matches(M, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.9444444444444444, 1.0, 0.9714285714285714, 0.0, 1.0)

time: 10.7 ms


In [130]:
#--------Rule-based matcher + triggers--------#
# 1. write rules
# 2. add triggers

time: 734 µs


In [131]:
# Create rule-based matcher and add rules.
rm = mg.BooleanRuleMatcher()
rm.add_rule(['title_title_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6' 
             ], 
            feature_table=feature_table)


True

time: 2.65 ms


In [132]:
rm.add_rule(['author_author_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.6'], 
            feature_table=f)
rm.add_rule(['binding_binding_jac_qgm_3_qgm_3(ltuple, rtuple) > 0.5'], 
            feature_table=f)

True

time: 3.16 ms


In [134]:
L.head()

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,gold,predicted
0,28,15368811,9780217110952,0.852941,1.000000,1,1.000000,1,1,1
1,11,5681971,9780073380339,0.807018,0.413793,0,0.200000,1,1,1
2,10,5514791,9780114958367,0.878049,0.000000,0,0.000000,0,0,0
3,41,10948704,9780131077744,0.333333,0.000000,0,0.000000,1,0,0
4,2,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1,1


time: 15.7 ms


In [137]:
# Note here the input is candidate set !!!
X = rm.predict(table=L, target_attr='predicted', 
               append=True, inplace=False)

time: 540 ms


In [138]:
X.head()

,_id,ltable.id,rtable.id,ltable.title,ltable.author,rtable.title,rtable.author,gold,predicted
0,0,4086892,9780226156439,The Bedtrick: Tales of Sex and Masquerade,"Doniger, Wendy",Bedtrick : Tales of Sex and Masquerade,"Doniger, Wendy",1,1
1,1,535599,9780060655808,HarperCollins Study Bible-NRSV,"Meeks, Wayne A.",Harpercollins Study Bible Standard Version wit...,"Meeks, Wayne A.",1,1
2,3,1386538,9780130901262,Adaptive Filter Theory,"Haykin, Simon S.",Adaptive Filter Theory,"Haykin, Simon",1,1
3,5,15280600,9780217455008,"Catalogue of Earthquakes on the Pacific Coast,...","McAdie, Alexander","Catalogue of Earthquakes on the Pacific Coast,...","Mcadie, Alexander",1,1
4,6,15286798,9780217825382,"Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill","Cured by an Incurable, by Crowquill and Pencil...","Pseud, Crowquill",1,1


time: 17.6 ms


In [139]:
# Evaluate rule-based matcher performance
eval_summary = mg.eval_matches(X, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7391304347826086, 0.9444444444444444, 0.8292682926829269, 2.0, 12.0)

time: 11.6 ms


In [140]:
mg.vis_debug_rm(rm, L, 'gold', feature_table)

time: 27.7 s


In [147]:
# address false positives
# Add triggers to address false negatives
# if authors match exactly then declare match
neg_trigger = mg.MatchTrigger()
neg_trigger.add_cond_rule(['lang_lang_lev(ltuple, rtuple) > 0.5'],
    feature_table=feature_table)
neg_trigger.add_cond_status(False)
neg_trigger.add_action(0)


True

time: 4.27 ms


In [148]:
Y = neg_trigger.execute(X, 'predicted', inplace=False)

time: 546 ms


In [149]:
eval_summary = mg.eval_matches(Y, 'gold', 'predicted')
(eval_summary['precision'], eval_summary['recall'], eval_summary['f1'],
 eval_summary['false_neg_num'], eval_summary['false_pos_num'])

(0.7727272727272727, 0.9444444444444444, 0.85, 2.0, 10.0)

time: 11.4 ms


In [56]:
# choose ml matcher + trigger

time: 773 µs


In [150]:
# Extract feature vectors using subset of features
N = mg.extract_feat_vecs(C, feature_table=f)

time: 329 ms


In [151]:
len(N)

68

time: 2.31 ms


In [157]:
dt.fit(table=G, exclude_attrs=['_id', 'ltable.id', 'rtable.id', 'gold'],
        target_attr='gold')

time: 5.89 ms


In [155]:
# Impute values
N.fillna(0, inplace=True)

time: 1.05 ms


In [158]:
O = dt.predict(table=N, exclude_attrs=['_id', 'ltable.id', 'rtable.id'],
              target_attr='predicted', append=True, inplace=False)

time: 8.82 ms


In [159]:
O.head()

,_id,ltable.id,rtable.id,title_title_jac_qgm_3_qgm_3,author_author_jac_qgm_3_qgm_3,binding_binding_jac_qgm_3_qgm_3,publisher_publisher_jac_qgm_3_qgm_3,lang_lang_lev,predicted
0,0,4086892,9780226156439,0.785714,1.000000,0,0.518519,1,1
1,1,535599,9780060655808,0.222222,1.000000,0,0.320000,1,1
2,2,12537042,9780030402432,0.243243,0.285714,0,0.025641,0,1
3,3,1386538,9780130901262,1.000000,0.785714,1,0.733333,1,1
4,4,13945932,9780205700424,0.375000,0.375000,0,0.733333,1,1


time: 13.4 ms


In [161]:
P = pos_trigger.execute(O, 'predicted', inplace=False)

time: 496 ms
